# PySpark Dataframes
For each instruction, type the appropriate code into the cell below the instruction. Then, run the code by pressing the `Run` button above.

1. Import the SparkSession class:

```
from pyspark.sql import SparkSession
```

In [1]:
from pyspark.sql import SparkSession

2. Use this class to instiate a Spark session:

```
spark = SparkSession \
    .builder \
    .appName("My First PySpark App") \
    .getOrCreate()
```

In [2]:
spark = SparkSession \
    .builder \
    .appName("My First PySpark App") \
    .getOrCreate()

3. Take a look at the session object:

```
spark
```

In [3]:
spark

4. Read the contents of a csv file into a Dataframe named 'accounts':

```
accounts = spark.read.option('header', 'true').csv('./data/accounts.csv')
```

In [4]:
accounts = spark.read.option('header', 'true').csv('./data/accounts.csv')
accounts.show(5)

+------------------+---------+-----------+----------+-------+
|    account_number|      aba|        bic|    opened|balance|
+------------------+---------+-----------+----------+-------+
|CBXD44669851029839|102317138|RWWEGBRO0T5|2009-06-04| 285769|
|GMHS11683964344077|082341703|   TIDCGB2I|2014-06-26| 591000|
|GCQT83212037197512|107809939|HGMJGB9AIFY|2006-08-07|1631126|
|TDDI73474064461675|046709967|VQEAGBT2HP7|2001-03-20|1977359|
|HDVG29919274654139|069217423|BQCSGBADD0Z|2014-11-05| 208204|
+------------------+---------+-----------+----------+-------+
only showing top 5 rows



5. Take a look at the Dataframe's schema:

```
accounts.printSchema()
```

In [5]:
accounts.printSchema()

root
 |-- account_number: string (nullable = true)
 |-- aba: string (nullable = true)
 |-- bic: string (nullable = true)
 |-- opened: string (nullable = true)
 |-- balance: string (nullable = true)



6. Read the contents of a parquet file into a variable:
```
transactions = spark.read.option('header', True).parquet('./data/transactions.parquet')
```

In [6]:
transactions = spark.read.option('header', 'true').parquet('./data/transactions.parquet')
transactions.show(5)

+------------------+------+----------+
|    account_number|amount|  datetime|
+------------------+------+----------+
|GSHP69974285041169| -9034|2022-09-08|
|LVTT31407534039456| -2952|2022-04-02|
|RJSO40371783238228|  3405|2022-05-02|
|UOIH17449189991636| -4049|2022-10-07|
|FBXP58394333953676| -4157|2022-06-22|
+------------------+------+----------+
only showing top 5 rows



7. See how many rows are in the new Dataframe:
```
transactions.count()
```

In [7]:
transactions.count()

1000000

8. Make a new Dataframe by grouping the transactions by account number and summing the groups. This will combine the transactions per account:

```
account_transactions = transactions.groupby('account_number').sum()
```

In [8]:
account_transactions = transactions.groupby('account_number').sum()
account_transactions.show(5)

+------------------+-----------+
|    account_number|sum(amount)|
+------------------+-----------+
|FBXK78425844480007|     -99434|
|XJIU55438863095422|      77947|
|XBYT37304125118047|      65101|
|UQSE17000937342665|     118473|
|KWOU43650129218895|     -35411|
+------------------+-----------+
only showing top 5 rows



9. Combine the accounts with the summed transaction values:

```
with_sum = accounts.join(account_transactions, 'account_number', 'inner')
```

In [9]:
with_sum = accounts.join(account_transactions, 'account_number', 'inner')
with_sum.show(5)

+------------------+---------+-----------+----------+-------+-----------+
|    account_number|      aba|        bic|    opened|balance|sum(amount)|
+------------------+---------+-----------+----------+-------+-----------+
|FBXK78425844480007|125056531|   TFJQGB5D|2014-05-07| 710027|     -99434|
|XJIU55438863095422|036174230|   KILTGB3K|2009-12-18| 339452|      77947|
|XBYT37304125118047|061333512|UVXHGB0SHJC|2013-12-21| 948840|      65101|
|UQSE17000937342665|028459374|JBPYGB7X207|2009-07-30| 928422|     118473|
|KWOU43650129218895|038163281|   PHLOGB1V|2016-06-14|1119190|     -35411|
+------------------+---------+-----------+----------+-------+-----------+
only showing top 5 rows



10. Get the current balance per account by summing the transaction sums with the initial account balance:

```
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))
```

In [10]:
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))
accounts.show(5)

+------------------+---------+-----------+----------+-------+-----------+-----------+
|    account_number|      aba|        bic|    opened|balance|sum(amount)|new_balance|
+------------------+---------+-----------+----------+-------+-----------+-----------+
|FBXK78425844480007|125056531|   TFJQGB5D|2014-05-07| 710027|     -99434|   610593.0|
|XJIU55438863095422|036174230|   KILTGB3K|2009-12-18| 339452|      77947|   417399.0|
|XBYT37304125118047|061333512|UVXHGB0SHJC|2013-12-21| 948840|      65101|  1013941.0|
|UQSE17000937342665|028459374|JBPYGB7X207|2009-07-30| 928422|     118473|  1046895.0|
|KWOU43650129218895|038163281|   PHLOGB1V|2016-06-14|1119190|     -35411|  1083779.0|
+------------------+---------+-----------+----------+-------+-----------+-----------+
only showing top 5 rows



11. Get accounts with negative current balances:

```
neg_balance = accounts.filter(accounts.new_balance < 0)
```

In [11]:
neg_balance = accounts.filter(accounts.new_balance < 0)
neg_balance.show(5)

+------------------+---------+-----------+----------+-------+-----------+-----------+
|    account_number|      aba|        bic|    opened|balance|sum(amount)|new_balance|
+------------------+---------+-----------+----------+-------+-----------+-----------+
|JMTP45763117901514|108480276|   AKJGGBSL|2000-11-08|  57948|     -85521|   -27573.0|
|RBUE05237750254383|054916920|   EXUPGBRO|2010-09-07|  35094|    -138553|  -103459.0|
|RJMY57096756148587|098748471|   SPIOGBOT|2005-05-11|  43690|    -102019|   -58329.0|
|ZYMB62177146259441|115359235|   XWITGB3O|2014-01-21|  26460|     -81891|   -55431.0|
|LRTH65732611614073|108298031|ZWIOGB1G1I8|2013-05-27|  50235|    -154066|  -103831.0|
+------------------+---------+-----------+----------+-------+-----------+-----------+
only showing top 5 rows



12. Read client data from a json file:

```
clients = spark.read.json('./data/clients.json')
```

In [12]:
clients = spark.read.json('./data/clients.json')
clients.show(5)

+------------------+--------------------+--------------------+----------+---------+
|    account_number|             address|               email|first_name|last_name|
+------------------+--------------------+--------------------+----------+---------+
|CBXD44669851029839|591 Christopher C...|dmorales@example.com|   Jeffrey|  Schultz|
|GMHS11683964344077|44967 Watson Cany...|lovebrian@example...|  Jennifer|   Waters|
|GCQT83212037197512|240 Ryan Heights\...| peter16@example.com|      Mark|  Marquez|
|TDDI73474064461675|505 Clark Field S...| randy08@example.org|  Clifford|    Young|
|HDVG29919274654139|9120 Olsen Curve\...|stevensdavid@exam...|   Jessica|    Price|
+------------------+--------------------+--------------------+----------+---------+
only showing top 5 rows



13. Get the clients with a negative balance:

```
clients = clients.join(neg_balance, 'account_number', 'inner')
```

In [13]:
clients = clients.join(neg_balance, 'account_number', 'inner')
clients.count()

118

14. Look at the top five clients with negative balances:

```
clients.select(['first_name', 'last_name', 'account_number', 'new_balance']).show(5)
```

In [14]:
clients.select(['first_name', 'last_name', 'account_number', 'new_balance']).show(5)

+----------+---------+------------------+-----------+
|first_name|last_name|    account_number|new_balance|
+----------+---------+------------------+-----------+
|    Meagan| Sandoval|JMTP45763117901514|   -27573.0|
|  Michelle|   Knight|RBUE05237750254383|  -103459.0|
|      Paul|   Massey|RJMY57096756148587|   -58329.0|
|  Michelle|    Perez|ZYMB62177146259441|   -55431.0|
|     David|    Green|LRTH65732611614073|  -103831.0|
+----------+---------+------------------+-----------+
only showing top 5 rows

